In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv
import re
import json

In [2]:
# inference_path = 'ctr_valid_inference_mixtral.pkl'
inference_path = 'zeroshot_inference_test.json'
with open(inference_path, 'rb') as f:
    inference_dict = json.load(f)
print(len(inference_dict))

1496


In [3]:
ratings_df = pd.read_csv('../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [4]:
user_valid_movie_rating_df = ratings_df.groupby('user_id').nth(-1)
user_valid_movie_rating_df.head(10)

,user_id,movie_id,rating
52,1,48,5
453,10,2252,5
529,100,265,3
613,1000,1104,5
990,1001,647,3
1056,1002,1222,5
1085,1003,337,4
1566,1004,1266,5
1658,1005,266,1
1702,1006,2762,4


In [5]:
user_test_movie_rating_df = ratings_df.groupby('user_id').nth(-1)
user_test_movie_rating_df.head(10)

,user_id,movie_id,rating
52,1,48,5
453,10,2252,5
529,100,265,3
613,1000,1104,5
990,1001,647,3
1056,1002,1222,5
1085,1003,337,4
1566,1004,1266,5
1658,1005,266,1
1702,1006,2762,4


In [6]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 0
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 120 -- No: 49 -- Invalid: 1327
invalid_users: ['1', '10', '100', '1000', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '1010', '1011', '1012', '1013', '1014', '1015', '1017', '1019', '102', '1020', '1021', '1023', '1024', '1026', '1027', '1028', '1029', '103', '1030', '1031', '1032', '1033', '1034', '1035', '1036', '1037', '1038', '1039', '1040', '1041', '1042', '1043', '1044', '1045', '1046', '1047', '1048', '1049', '105', '1050', '1051', '1052', '1053', '1054', '1055', '1056', '1057', '1058', '1060', '1061', '1062', '1063', '1064', '1065', '1066', '1067', '1068', '1069', '107', '1070', '1071', '1072', '1073', '1074', '1075', '1076', '1077', '108', '1080', '1081', '1082', '1083', '1084', '1085', '1086', '1087', '1088', '1089', '1090', '1091', '1092', '1093', '1096', '1097', '1099', '11', '110', '1100', '1101', '1102', '1103', '1104', '1106', '1107', '1108', '1109', '111', '1110', '1111', '1112', '1113', '1114', '1115', '1116', '1119', '112', '1120', '1121', '112

In [9]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    # original_rating = user_valid_movie_rating_df[user_valid_movie_rating_df['user_id'] == user]['rating'].values[0]
    try:
        original_rating = user_test_movie_rating_df[user_test_movie_rating_df['user_id'] == user]['rating'].values[0]
    except:
        continue
    # print(original_rating)
    if original_rating >= 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[]


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [13]:
movies_df = pd.read_csv('../movie_summary_25_words.csv')
movies_df[movies_df.movie_id == 1466]

,movie_id,name,cleaned_genre,year,summary
1439,1466,Donnie Brasco,"Crime,Drama",1997,"""Donnie Brasco"" is a gripping crime drama base..."


In [9]:
ctr_valid_dataset_path = './ctr_zeroshot_dataset/ctr_valid_zeroshot_dataset.pkl'
with open(ctr_valid_dataset_path, 'rb') as f:
    ctr_valid_data_dict = pickle.load(f)
print(len(ctr_valid_data_dict))

6040


In [14]:
print(ctr_valid_data_dict[1005])


    You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
    User Profile -
    The user is a male lawyer between 35 to 44 years of age
            The user has a diverse taste in movies, with a preference for those that are thought-provoking and emotionally impactful. They enjoy movies that explore complex themes and blur the lines between reality and fantasy, as seen in their appreciation for "Being John Malkovich" and "Bowfinger." The user also has a soft spot for classic Christmas films, as evidenced by their fondness for "Miracle on 34th Street."
            
            In terms of genres, the user leans towards drama and comedy, with a particular interest in coming-of-age stories and character-driven narratives. They are drawn to movies that feature strong performances and explore the complexities of human relationships, as seen in their appreciation for "Diner" and "Schi

In [19]:
for user in invalid_users:
    print(user, inference_dict[user])

1164 
To analyze the user's behavior and predict whether they will like "Who Framed Roger Rabbit?" or not, we need to consider their movie watching history and preferences.

The user tends to enjoy movies that are character-driven, thought-provoking, and emotionally resonant. They have a clear preference for romantic comedies, comedies with wit and humor, and dramas that explore complex themes such as self-discovery, relationships, and the human condition.

Looking at the user's movie watching history, we can see that they have enjoyed movies such as "Mickey Blue Eyes," "The Mighty," "Out of Africa," "Rushmore," "Sleepy Hollow," "The Talented Mr. Ripley," "The Thomas Crown Affair," "Velvet Goldmine," and "Where the Heart Is." These movies all share common themes of love, relationships, and personal growth, with a strong focus on character development and storytelling.

On the other hand, the user has disliked movies such as "Midsummer Night's Dream," "Mystery Men," "Pushing Tin," "Stir